<b> Script para rodar simulação paramétrica de HVAC para benchmarking energético <b>

0. Ler bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from eppy import modeleditor
from eppy.modeleditor import IDF
from eppy.results import readhtml, fasthtml

1. Configurações iniciais:

In [2]:
# Configurações iniciais do EPPY:
iddfile = "C:\EnergyPlusV9-4-0\Energy+.idd"


4. Ler HTML e buscar e fazer o dataframe com Zones, Andares e UserDesignLoad

In [42]:
# Lendo HTML:

files = (1,2,3,4,5,6,7,8)
lotes = (1,2)
cidades = ['Curitiba', 'Pelotas', 'Sao_Paulo', 'Brasilia','Santos','Goiania','Cuiaba','Sao_Luis']

for file in files:
    fname1 = 'LOTE_1/archertype_office_40_case'+str(file)+'.idf'
    IDF.setiddname(iddfile)
    idf1 = IDF(fname1)
    fname = 'LOTE_1/Sao_Luis/archertype_office_40_case'+str(file)+'_1Table.html'
    filehandle = open(fname, 'r').read()
    htables = readhtml.titletable(filehandle)
    firstitem = htables[0]
    import pprint
    pp = pprint.PrettyPrinter()

    filehandle = open(fname, 'r') # get a file handle to the html file
    namedtable = fasthtml.tablebyname(filehandle, "Zone Sensible Cooling")

    # Declarando variáveis
    zones = len(namedtable[1])
    zone_names = []
    zone_loads = []

    # Listar as zonas térmicas:
    for n in range(zones):
      zone_names.append(namedtable[1][n][0])
    del zone_names[0]

    # Listar o User Design Loads:
    for n in range(zones):
      zone_loads.append(namedtable[1][n][2])

    del zone_loads[0]

    zones = []
    for n in range(len(zone_names)):
      zones.append(zone_names[n][-1])

    floors = []
    for n in range(len(zone_names)):
      if zone_names[n][0] == "G":
        floors.append(10)
      else:
        floors.append(zone_names[n][0])

    data = {
        'zones': zones,
        'floor': floors,
        'loads': zone_loads
      }

    #Lendo a lista de HVAC systems
    condenser_db = pd.read_csv("DADOS_HVAC\condenser.csv", sep=',')
    evaporator_db = pd.read_csv("DADOS_HVAC\evaporator.csv", sep=',')

    cooling_list_evaporator = evaporator_db['Cooling Coil Gross Rated Total Capacity [W]']
    cooling_list_evaporator = cooling_list_evaporator.tolist()

    cooling_list_condenser = condenser_db['Gross Rated Total Cooling Capacity [W]']
    cooling_list_condenser = cooling_list_condenser.tolist()

    # Selecionando a evaporadora:

    my_evaporator = []

    for n in range(len(data['loads'])):

        for x in cooling_list_evaporator:
            
            if x >= data['loads'][n]:
                my_evaporator.append(x)
                break

    #  Printar a evaporadora para conferir:
    #  print(my_evaporator)

    # Selecionando a condensadora:

    my_condenser = []
    user_design_load_floors = []

    data = pd.DataFrame.from_dict(data)

    for f in data['floor']:
        user_design_load_floors.append(data.loc[data['floor']==f]['loads'].sum())

    for h in range(len(user_design_load_floors)):
        for x in cooling_list_condenser:
            if x >= user_design_load_floors[h]:
                my_condenser.append(x)
                break

    data = {
        'zones': zones,
        'floor': floors,
        'loads': zone_loads,
        'condenser': my_condenser,
        'evaporator': my_evaporator
      }

    data = pd.DataFrame(data)

    # Identificando todas as variáveis da evaporadora:

    my_evaporator = []
    data['zones'] = pd.to_numeric(data['zones'])
    data['floor'] = pd.to_numeric(data['floor'])
    number_of_zones = list(dict.fromkeys(data['zones']))
    number_of_floors = list(dict.fromkeys(data['floor']))


    for m in number_of_floors:
        for z in number_of_zones:
            evaporator_1 = data.loc[(data['floor']==m)&(data['zones']==z)]['evaporator'].mean()
            a = evaporator_db.loc[evaporator_db['Cooling Coil Gross Rated Total Capacity [W]']==evaporator_1]['Cooling Supply Air Flow Rate [m3/s]'].tolist()
            b = evaporator_db.loc[evaporator_db['Cooling Coil Gross Rated Total Capacity [W]']==evaporator_1]['Heating Supply Air Flow Rate [m3/s]'].tolist()
            c = evaporator_db.loc[evaporator_db['Cooling Coil Gross Rated Total Capacity [W]']==evaporator_1]['Supply Fan Delta Pressure [Pa]'].tolist()
            d = evaporator_db.loc[evaporator_db['Cooling Coil Gross Rated Total Capacity [W]']==evaporator_1]['Supply Fan Total Efficiency'].tolist()
            e = evaporator_db.loc[evaporator_db['Cooling Coil Gross Rated Total Capacity [W]']==evaporator_1]['Cooling Coil Gross Rated Total Capacity [W]'].tolist()
            f = evaporator_db.loc[evaporator_db['Cooling Coil Gross Rated Total Capacity [W]']==evaporator_1]['Heating Coil Gross Rated Total Capacity [W]'].tolist()
            g = evaporator_db.loc[evaporator_db['Cooling Coil Gross Rated Total Capacity [W]']==evaporator_1]['Supply Fan Power [W]'].tolist()
            x = [a,b,c,d,e,f,g]
            my_evaporator.append(x)

    my_evaporator = pd.DataFrame(my_evaporator, columns= ['Cooling Supply Air Flow Rate [m3/s]', 'Heating Supply Air Flow Rate [m3/s]','Supply Fan Delta Pressure [Pa]','Supply Fan Total Efficiency','Cooling Coil Gross Rated Total Capacity [W]','Heating Coil Gross Rated Total Capacity [W]','Supply Fan Power [W]'])

    # Editar o objeto HVAC Fan:

    z = 0
    for a in range(len(zone_names)):
        hvac_1 = idf1.idfobjects['HVACTEMPLATE:ZONE:VRF'][z]
        hvac_1.Cooling_Coil_Gross_Rated_Total_Capacity = my_evaporator['Cooling Coil Gross Rated Total Capacity [W]'][z][0]
        hvac_1.Heat_Pump_Heating_Coil_Gross_Rated_Capacity = my_evaporator['Heating Coil Gross Rated Total Capacity [W]'][z][0]
        hvac_1.Heating_Supply_Air_Flow_Rate = my_evaporator['Heating Supply Air Flow Rate [m3/s]'][z][0]
        hvac_1.Cooling_Supply_Air_Flow_Rate = my_evaporator['Cooling Supply Air Flow Rate [m3/s]'][z][0]
        hvac_1.Supply_Fan_Delta_Pressure = my_evaporator['Supply Fan Delta Pressure [Pa]'][z][0]
        hvac_1.Supply_Fan_Motor_Efficiency = my_evaporator['Supply Fan Total Efficiency'][z][0]
        z = z + 1


    # Identificando todas as variáveis da condensadora:

    # para o pavimento GF:

    condenser_10 = data.loc[data['floor']==10]['condenser'].mean()
    condenser_10 = condenser_db.loc[condenser_db['Gross Rated Total Cooling Capacity [W]'] == condenser_10]

    my_cooling_10 = condenser_10['Gross Rated Total Cooling Capacity [W]']
    my_heating_10 = condenser_10['Gross Rated Total Heating Capacity [W]']
    my_heating_cop_10 = condenser_10['Gross Rated Heating COP [W/W]']
    my_cooling_cop_10 = condenser_10['Gross Rated Cooling COP [W/W]']

    # para o pavimento 1:

    condenser_1 = data.loc[data['floor']==1]['condenser'].mean()
    condenser_1 = condenser_db.loc[condenser_db['Gross Rated Total Cooling Capacity [W]'] == condenser_1]

    my_cooling_1 = condenser_1['Gross Rated Total Cooling Capacity [W]']
    my_heating_1 = condenser_1['Gross Rated Total Heating Capacity [W]']
    my_heating_cop_1 = condenser_1['Gross Rated Heating COP [W/W]']
    my_cooling_cop_1 = condenser_1['Gross Rated Cooling COP [W/W]']

    # para o pavimento 2:

    condenser_2 = data.loc[data['floor']==2]['condenser'].mean()
    condenser_2 = condenser_db.loc[condenser_db['Gross Rated Total Cooling Capacity [W]'] == condenser_2]

    my_cooling_2 = condenser_2['Gross Rated Total Cooling Capacity [W]']
    my_heating_2 = condenser_2['Gross Rated Total Heating Capacity [W]']
    my_heating_cop_2 = condenser_2['Gross Rated Heating COP [W/W]']
    my_cooling_cop_2 = condenser_2['Gross Rated Cooling COP [W/W]']

    # para o pavimento 3:

    condenser_3 = data.loc[data['floor']==3]['condenser'].mean()
    condenser_3 = condenser_db.loc[condenser_db['Gross Rated Total Cooling Capacity [W]'] == condenser_3]

    my_cooling_3 = condenser_3['Gross Rated Total Cooling Capacity [W]']
    my_heating_3 = condenser_3['Gross Rated Total Heating Capacity [W]']
    my_heating_cop_3 = condenser_3['Gross Rated Heating COP [W/W]']
    my_cooling_cop_3 = condenser_3['Gross Rated Cooling COP [W/W]']

    # para o pavimento 4:

    condenser_4 = data.loc[data['floor']==4]['condenser'].mean()
    condenser_4 = condenser_db.loc[condenser_db['Gross Rated Total Cooling Capacity [W]'] == condenser_4]

    my_cooling_4 = condenser_4['Gross Rated Total Cooling Capacity [W]']
    my_heating_4 = condenser_4['Gross Rated Total Heating Capacity [W]']
    my_heating_cop_4 = condenser_4['Gross Rated Heating COP [W/W]']
    my_cooling_cop_4 = condenser_4['Gross Rated Cooling COP [W/W]']

    # para o pavimento 5:

    condenser_5 = data.loc[data['floor']==5]['condenser'].mean()
    condenser_5 = condenser_db.loc[condenser_db['Gross Rated Total Cooling Capacity [W]'] == condenser_5]

    my_cooling_5 = condenser_5['Gross Rated Total Cooling Capacity [W]']
    my_heating_5 = condenser_5['Gross Rated Total Heating Capacity [W]']
    my_heating_cop_5 = condenser_5['Gross Rated Heating COP [W/W]']
    my_cooling_cop_5 = condenser_5['Gross Rated Cooling COP [W/W]']

    # para o pavimento 6:

    condenser_6 = data.loc[data['floor']==6]['condenser'].mean()
    condenser_6 = condenser_db.loc[condenser_db['Gross Rated Total Cooling Capacity [W]'] == condenser_6]

    my_cooling_6 = condenser_6['Gross Rated Total Cooling Capacity [W]']
    my_heating_6 = condenser_6['Gross Rated Total Heating Capacity [W]']
    my_heating_cop_6 = condenser_6['Gross Rated Heating COP [W/W]']
    my_cooling_cop_6 = condenser_6['Gross Rated Cooling COP [W/W]']

    # para o pavimento 7:

    condenser_7 = data.loc[data['floor']==7]['condenser'].mean()
    condenser_7 = condenser_db.loc[condenser_db['Gross Rated Total Cooling Capacity [W]'] == condenser_7]

    my_cooling_7 = condenser_7['Gross Rated Total Cooling Capacity [W]']
    my_heating_7 = condenser_7['Gross Rated Total Heating Capacity [W]']
    my_heating_cop_7 = condenser_7['Gross Rated Heating COP [W/W]']
    my_cooling_cop_7 = condenser_7['Gross Rated Cooling COP [W/W]']

    # para o pavimento 8:

    condenser_8 = data.loc[data['floor']==8]['condenser'].mean()
    condenser_8 = condenser_db.loc[condenser_db['Gross Rated Total Cooling Capacity [W]'] == condenser_8]

    my_cooling_8 = condenser_8['Gross Rated Total Cooling Capacity [W]']
    my_heating_8 = condenser_8['Gross Rated Total Heating Capacity [W]']
    my_heating_cop_8 = condenser_8['Gross Rated Heating COP [W/W]']
    my_cooling_cop_8 = condenser_8['Gross Rated Cooling COP [W/W]']

    # para o pavimento 9:

    condenser_9 = data.loc[data['floor']==9]['condenser'].mean()
    condenser_9 = condenser_db.loc[condenser_db['Gross Rated Total Cooling Capacity [W]'] == condenser_9]

    my_cooling_9 = condenser_9['Gross Rated Total Cooling Capacity [W]']
    my_heating_9 = condenser_9['Gross Rated Total Heating Capacity [W]']
    my_heating_cop_9 = condenser_9['Gross Rated Heating COP [W/W]']
    my_cooling_cop_9 = condenser_9['Gross Rated Cooling COP [W/W]']

    # Editar o objeto de HVAC Cooling Load SYSTEM

    # pavimento GF:

    system_10 = idf1.idfobjects['HVACTEMPLATE:SYSTEM:VRF'][0]

    system_10.Gross_Rated_Total_Cooling_Capacity = my_cooling_10
    system_10.Gross_Rated_Heating_Capacity = my_heating_10
    system_10.Gross_Rated_Cooling_COP = my_cooling_cop_10
    system_10.Gross_Rated_Heating_COP = my_heating_cop_10
    system_10.Heat_Pump_Waste_Heat_Recovery = 'Yes'

    # pavimento 1:

    system_1 = idf1.idfobjects['HVACTEMPLATE:SYSTEM:VRF'][1]

    system_1.Gross_Rated_Total_Cooling_Capacity = my_cooling_1
    system_1.Gross_Rated_Heating_Capacity = my_heating_1
    system_1.Gross_Rated_Cooling_COP = my_cooling_cop_1
    system_1.Gross_Rated_Heating_COP = my_heating_cop_1
    system_1.Heat_Pump_Waste_Heat_Recovery = 'Yes'

    # pavimento 2:

    system_2 = idf1.idfobjects['HVACTEMPLATE:SYSTEM:VRF'][2]

    system_2.Gross_Rated_Total_Cooling_Capacity = my_cooling_2
    system_2.Gross_Rated_Heating_Capacity = my_heating_2
    system_2.Gross_Rated_Cooling_COP = my_cooling_cop_2
    system_2.Gross_Rated_Heating_COP = my_heating_cop_2
    system_2.Heat_Pump_Waste_Heat_Recovery = 'Yes'

    # pavimento 3:

    system_3 = idf1.idfobjects['HVACTEMPLATE:SYSTEM:VRF'][3]

    system_3.Gross_Rated_Total_Cooling_Capacity = my_cooling_3
    system_3.Gross_Rated_Heating_Capacity = my_heating_3
    system_3.Gross_Rated_Cooling_COP = my_cooling_cop_3
    system_3.Gross_Rated_Heating_COP = my_heating_cop_3
    system_3.Heat_Pump_Waste_Heat_Recovery = 'Yes'

    # pavimento 4:

    system_4 = idf1.idfobjects['HVACTEMPLATE:SYSTEM:VRF'][4]

    system_4.Gross_Rated_Total_Cooling_Capacity = my_cooling_4
    system_4.Gross_Rated_Heating_Capacity = my_heating_4
    system_4.Gross_Rated_Cooling_COP = my_cooling_cop_4
    system_4.Gross_Rated_Heating_COP = my_heating_cop_4
    system_4.Heat_Pump_Waste_Heat_Recovery = 'Yes'

    # pavimento 5:

    system_5 = idf1.idfobjects['HVACTEMPLATE:SYSTEM:VRF'][5]

    system_5.Gross_Rated_Total_Cooling_Capacity = my_cooling_5
    system_5.Gross_Rated_Heating_Capacity = my_heating_5
    system_5.Gross_Rated_Cooling_COP = my_cooling_cop_5
    system_5.Gross_Rated_Heating_COP = my_heating_cop_5
    system_5.Heat_Pump_Waste_Heat_Recovery = 'Yes'

    # pavimento 6:

    system_6 = idf1.idfobjects['HVACTEMPLATE:SYSTEM:VRF'][6]

    system_6.Gross_Rated_Total_Cooling_Capacity = my_cooling_6
    system_6.Gross_Rated_Heating_Capacity = my_heating_6
    system_6.Gross_Rated_Cooling_COP = my_cooling_cop_6
    system_6.Gross_Rated_Heating_COP = my_heating_cop_6
    system_6.Heat_Pump_Waste_Heat_Recovery = 'Yes'

    # pavimento 7:

    system_7 = idf1.idfobjects['HVACTEMPLATE:SYSTEM:VRF'][7]

    system_7.Gross_Rated_Total_Cooling_Capacity = my_cooling_7
    system_7.Gross_Rated_Heating_Capacity = my_heating_7
    system_7.Gross_Rated_Cooling_COP = my_cooling_cop_7
    system_7.Gross_Rated_Heating_COP = my_heating_cop_7
    system_7.Heat_Pump_Waste_Heat_Recovery = 'Yes'

    # pavimento 8:

    system_8 = idf1.idfobjects['HVACTEMPLATE:SYSTEM:VRF'][8]

    system_8.Gross_Rated_Total_Cooling_Capacity = my_cooling_8
    system_8.Gross_Rated_Heating_Capacity = my_heating_8
    system_8.Gross_Rated_Cooling_COP = my_cooling_cop_8
    system_8.Gross_Rated_Heating_COP = my_heating_cop_8
    system_8.Heat_Pump_Waste_Heat_Recovery = 'Yes'

    # pavimento 9:

    system_9 = idf1.idfobjects['HVACTEMPLATE:SYSTEM:VRF'][9]

    system_9.Gross_Rated_Total_Cooling_Capacity = my_cooling_9
    system_9.Gross_Rated_Heating_Capacity = my_heating_9
    system_9.Gross_Rated_Cooling_COP = my_cooling_cop_9
    system_9.Gross_Rated_Heating_COP = my_heating_cop_9
    system_9.Heat_Pump_Waste_Heat_Recovery = 'Yes'

    # Criar o IDF Modificado:

    idf1.saveas('LOTE_1/Sao_Luis/LOTE_1_Sao_Luis_case_'+str(file)+'.idf')